In [7]:
#import dependencies
import pandas as pd
import os

# Load files
school_data_csv = "Resources/schools_complete.csv"
student_data_csv = "Resources/students_complete.csv"

#Read in and store files in dataframe
school_data = pd.read_csv(school_data_csv)
student_data = pd.read_csv(student_data_csv)

#Merge data 
complete_data = pd.merge(school_data, student_data, how="left", on=["school_name", "school_name"])
complete_data.head()


,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


 ## District Summary

In [8]:
#count,print and save data from the school data
school_names = school_data['school_name'].count()
print(school_names)
#count,print and save data from the student data
number_students = student_data['student_name'].count()
print(number_students)
#sum and save the budget
total_budget = school_data['budget'].sum()
print(total_budget)
#calculate average for the math and reading scores
avg_math = student_data['math_score'].mean()
print(avg_math)
avg_reading = student_data['reading_score'].mean()
print(avg_reading)
#calculate the math and reading scores
math_passing = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
print(math_passing) 
percent_passed_math = (math_passing/number_students) * 100
print(percent_passed_math)
reading_passing = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
print(reading_passing)
percent_passed_reading = (reading_passing/number_students) * 100
print(percent_passed_reading)


#calculate overall passing grade score
avg_passing_overall =  (student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]['student_name'].count()/number_students) * 100 
print(avg_passing_overall)

   
                    
                       
                       
                       
#create dataframe to hold results

district_summary = pd.DataFrame({
        "Total Schools" : [school_names],
        "Total Students" : [number_students],
        "Total Budget" : [total_budget],
        "Average Math Score" : [avg_math],
        "Average Reading Score" : [avg_reading],
        "% Passing Math" : [percent_passed_math],
        "% Passing Reading" : [percent_passed_reading],
        "% Overall Reading" : [avg_passing_overall]
})
district_summary
#format percentages, decimal points,  and dollar signs
district_summary.style.format({"Total Students": "{:,}", "Total Budget": "${:,.2f}"})






15
39170
24649428
78.98537145774827
81.87784018381414
29370
74.9808526933878
33610
85.80546336482001
65.17232575950983


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Reading
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

In [9]:
#group by school
school_grouped = complete_data.groupby(['school_name'])
#seperate the school types
school_types = school_data.set_index('school_name')['type']
#count the total number of students
total_students_per_school = complete_data["school_name"].value_counts()
#find the total budget
budget_per_school = complete_data.groupby(["school_name"]).mean()["budget"]
#calculate budget
budget_per_student = (budget_per_school/total_students_per_school)

#calculate average math and reading scores
avg_math_score = school_grouped['math_score'].mean()
avg_reading_score = school_grouped['reading_score'].mean()

#calculate totals for passing math/reading per school
math_passing2 = complete_data[complete_data['math_score'] >=70].groupby('school_name').count()
math_passing_final = (math_passing2["math_score"]/total_students_per_school) * 100
read_passing2 = complete_data[complete_data['reading_score'] >=70].groupby('school_name').count()
read_passing_final = (read_passing2["reading_score"]/total_students_per_school) * 100
overall_passing = complete_data[(complete_data['reading_score'] >= 70) & (complete_data['math_score'] >= 70)].groupby('school_name')['Student ID'].count()
overall_passing_final = (overall_passing/total_students_per_school) * 100


school_summary = pd.DataFrame({
                "School Type" : school_types,
                "Total Students " : total_students_per_school,
                "Total School Budget" : budget_per_school,
                "Per Student Budget" : budget_per_student,
                "Average Math Score" : avg_math_score,
                "Average Reading Score" : avg_reading_score,
                "% Passing Math" : math_passing_final,
                "% Passing Reading" : read_passing_final,
                "% Overall Passing" : overall_passing_final,
})

#format percentage, dollar signs, and decimals
school_summary.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools

In [10]:
#sort the values by the passing rate then printing out the head
top_5_schools = school_summary.sort_values("% Overall Passing", ascending = False)
top_5_schools.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})
top_5_schools.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools

In [11]:
bottom_5_schools = school_summary.sort_values("% Overall Passing", ascending = True)
bottom_5_schools.style.format({"Total School Budget" : "${:,.2f}", "Per Student Budget" : "${:,.2f}"})
bottom_5_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Score

In [12]:
#calculate average math scores
ms_9 = complete_data.loc[complete_data["grade"] == "9th"].groupby(["school_name"])["math_score"].mean()
ms_10 = complete_data.loc[complete_data["grade"] == "10th"].groupby(["school_name"])["math_score"].mean()
ms_11 = complete_data.loc[complete_data["grade"] == "11th"].groupby(["school_name"])["math_score"].mean()
ms_12 = complete_data.loc[complete_data["grade"] == "12th"].groupby(["school_name"])["math_score"].mean()

#create dataframe
math_grade_summary = pd.DataFrame({"9th" : ms_9,
                                  "10th" : ms_10,
                                  "11th" : ms_11,
                                  "12th" : ms_12})

math_grade_summary[["9th", "10th", "11th", "12th"]]

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score

In [13]:
#collect avg reading scores by grade
rs_9 = complete_data.loc[complete_data["grade"] == "9th"].groupby(["school_name"])["reading_score"].mean()
rs_10 = complete_data.loc[complete_data["grade"] == "10th"].groupby(["school_name"])["reading_score"].mean()
rs_11 = complete_data.loc[complete_data["grade"] == "11th"].groupby(["school_name"])["reading_score"].mean()
rs_12 = complete_data.loc[complete_data["grade"] == "12th"].groupby(["school_name"])["reading_score"].mean()

# create dataframe
reading_grade_summary = pd.DataFrame({"9th": rs_9,
      "10th": rs_10,
      "11th": rs_11,
      "12th": rs_12})

reading_grade_summary[["9th", "10th", "11th", "12th"]]

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by Spending

In [17]:
# create bins for the range of spending
spending_bins_amount = [0, 585, 630, 645, 680]
spending_bins_names = ["<$585", "$585-630", "$630-645", "$645-680"]

#create dataframe for the bins
school_summary["Spending Ranges (Per Student)"] = pd.cut(budget_per_student, spending_bins_amount,labels = spending_bins_names)
school_summary

#calculate mean scores for the bins
math_score_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"],2)
reading_score_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"],2)
math_pass_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"],2)
reading_pass_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"],2)
overall_pass_rate_spending = (math_pass_spending + reading_pass_spending)/2

#place values into a dataframe
scores_spending = pd.DataFrame({"Average Math Score":math_score_spending,
                                     "Average Reading Score":reading_score_spending,
                                     "% Passing Math":math_pass_spending,
                                     "% Passing Reading":reading_pass_spending ,
                                     "% Overall Passing":overall_pass_rate_spending})
scores_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,95.035
$585-630,81.90,83.16,87.13,92.72,89.925
$630-645,78.52,81.62,73.48,84.39,78.935
$645-680,77.00,81.03,66.16,81.13,73.645


## Scores by School Size

In [18]:
#create school size range bins
size_bins_amount = [0, 1000, 2000, 5000]
size_bins_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
#create dataframe for the bins
school_summary["School Size"] = pd.cut(total_students_per_school,size_bins_amount,labels = size_bins_names)
school_summary
#calculate the mean scores for the bins
math_score_size = round(school_summary.groupby(["School Size"]).mean()["Average Math Score"],2)
reading_score_size = round(school_summary.groupby(["School Size"]).mean()["Average Reading Score"],2)
math_pass_size = round(school_summary.groupby(["School Size"]).mean()["% Passing Math"],2)
reading_pass_size = round(school_summary.groupby(["School Size"]).mean()["% Passing Reading"],2)
overall_pass_rate_size = (math_pass_size + reading_pass_size)/2

#create a dataframe that renames the corresponding bins
scores_size = pd.DataFrame({"Average Math Score":math_score_size,
                                     "Average Reading Score":reading_score_size,
                                     "% Passing Math":math_pass_size,
                                     "% Passing Reading":reading_pass_size,
                                     "% Overall Passing":overall_pass_rate_size})
scores_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.825
Medium (1000-2000),83.37,83.86,93.60,96.79,95.195
Large (2000-5000),77.75,81.34,69.96,82.77,76.365


## Scores by School Type

In [19]:
math_score_type = school_summary.groupby(["School Type"]).mean()["Average Math Score"]
reading_score_type = school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
pass_math_type = school_summary.groupby(["School Type"]).mean()["% Passing Math"]
pass_reading_type = school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
overall_passing_rate_type = school_summary.groupby(["School Type"]).mean()["% Overall Passing"]

#create a dataframe that renames the corresponding bins
scores_type = pd.DataFrame({"Average Math Score" :math_score_type,
                           "Average Reading Score" :reading_score_type,
                           "% Passing Math" :pass_math_type,
                           "% Passing Reading" :pass_reading_type,
                           "% Overall Passing" :overall_passing_rate_type})
scores_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
